In [55]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
import warnings
warnings.filterwarnings('ignore')

# 1. CARGA LIMPIA
try:
    df = pd.read_parquet("data_cleaned/master_with_clusters.parquet")
except:
    df = pd.read_parquet("../data_cleaned/master_with_clusters.parquet")

df['Fecha'] = pd.to_datetime(df['Fecha'])

# 2. DEFINICIÓN DE QUINCENAS (Lógica de Negocio)
df['year'] = df['Fecha'].dt.year
df['month'] = df['Fecha'].dt.month
df['day'] = df['Fecha'].dt.day
# Quincena 1 (1-15), Quincena 2 (16-Fin)
df['quincena_num'] = np.where(df['day'] <= 15, 1, 2)

# 3. EFECTO CALENDARIO (Días de Pago)
# Calculamos cuántos días de "pago fuerte" cayeron en esa quincena
dates_ref = df[['Fecha', 'year', 'month', 'quincena_num']].drop_duplicates()
dates_ref['es_quincena'] = dates_ref['Fecha'].dt.day.isin([15, 30, 31, 1, 25, 26, 27, 28, 29]).astype(int)
payday_impact = dates_ref.groupby(['year', 'month', 'quincena_num'])['es_quincena'].sum().reset_index()
payday_impact.columns = ['year', 'month', 'quincena_num', 'dias_pago_periodo']

# 4. AGRUPACIÓN BASE (Target = Ingreso)
df_base = df.groupby(['year', 'month', 'quincena_num', 'Categoría']).agg({
    'ingreso': 'sum',
    'Cantidad': 'sum',
    'Precio_Unitario': 'mean',
    'Fecha': 'max' # Para referencia temporal
}).reset_index()

# Unir calendario
df_base = df_base.merge(payday_impact, on=['year', 'month', 'quincena_num'], how='left')

print(f"Dataset Quincenal Base: {df_base.shape}")
print(df_base.head(3))

Dataset Quincenal Base: (180, 9)
   year  month  quincena_num            Categoría  ingreso  Cantidad  \
0  2024      1             2           Congelados   123.60         8   
1  2024      1             2  Galletitas y Snacks    31.44         6   
2  2024      1             2              Lácteos    83.39         9   

   Precio_Unitario      Fecha  dias_pago_periodo  
0        15.450000 2024-01-31                  1  
1         5.240000 2024-01-31                  1  
2         9.363333 2024-01-31                  1  


In [56]:
def create_features(df):
    df = df.copy()
    # Ordenar CRONOLÓGICAMENTE para que los lags tengan sentido
    df = df.sort_values(['Categoría', 'year', 'month', 'quincena_num'])
    
    g = df.groupby('Categoría')
    
    # 1. TICKET PROMEDIO (Intensity)
    df['ticket_real'] = df['ingreso'] / (df['Cantidad'] + 1)
    df['lag_ticket_1'] = g['ticket_real'].shift(1) # ¿Cómo estuvo el ticket la quincena pasada?
    
    # 2. PRECIO RELATIVO (Inflation/Discount)
    # Precio actual vs promedio móvil de 2 meses (4 quincenas)
    df['precio_movil'] = g['Precio_Unitario'].transform(lambda x: x.shift(1).rolling(4).mean())
    df['indice_precio'] = df['Precio_Unitario'] / (df['precio_movil'] + 1e-6)
    
    # 3. LAGS DE INGRESO (History)
    # Lag 1 (Quincena anterior), Lag 2 (Mes anterior), Lag 4 (2 Meses)
    for lag in [1, 2, 4]:
        df[f'lag_ingreso_{lag}'] = g['ingreso'].shift(lag)
        
    # 4. TENDENCIA (Momentum)
    # Promedio móvil de ingresos (últimas 2 quincenas)
    df['rolling_ingreso_2q'] = g['ingreso'].transform(lambda x: x.shift(1).rolling(2).mean())
    
    # 5. CONTEXTO GLOBAL (Store Level)
    # ¿Cómo le fue a TODA la tienda la quincena pasada?
    # (Esto se calcula temporalmente y se une)
    return df

# Aplicar features
df_features = create_features(df_base)

# Contexto Global (Tienda Total)
store_total = df_features.groupby(['year', 'month', 'quincena_num'])['ingreso'].sum().reset_index()
store_total['store_lag_1'] = store_total['ingreso'].shift(1) # Lag global
df_features = df_features.merge(store_total[['year', 'month', 'quincena_num', 'store_lag_1']], 
                                on=['year', 'month', 'quincena_num'], how='left')

# Limpieza de Nulos generados por Lags
df_clean = df_features.dropna().reset_index(drop=True)

# One-Hot Encoding
df_final = pd.get_dummies(df_clean, columns=['Categoría'], prefix='Cat')

print(f"Dataset Final Listo: {df_final.shape}")

Dataset Final Listo: (148, 25)


In [57]:
# 1. DEFINICIÓN DE COLUMNAS
target = 'ingreso'
cols_excluir = [
    'ingreso', 'Cantidad', 'ticket_real', 'Precio_Unitario', # Leakage (Respuestas)
    'Fecha', 'year', 'month', 'day', 'quincena_num', 'periodo_id' # IDs
]
# Seleccionar solo numéricas y quitar las prohibidas
valid_features = df_final.select_dtypes(include=[np.number]).columns.tolist()
features = [c for c in valid_features if c not in cols_excluir]

# 2. SPLIT TEMPORAL
# Usamos las últimas 4 quincenas (2 meses) como test
split_date = df_final['Fecha'].max() - pd.Timedelta(weeks=8)

train_df = df_final[df_final['Fecha'] <= split_date].reset_index(drop=True)
test_df = df_final[df_final['Fecha'] > split_date].reset_index(drop=True)

# 3. PREPARACIÓN DE MATRICES (CRÍTICO: ALINEACIÓN)
X_train = train_df[features]
y_train_log = np.log1p(train_df[target]) # Log Transform

X_test = test_df[features]
y_test_real = test_df[target] # En pesos reales para validar

print("--- VERIFICACIÓN DE DIMENSIONES ---")
print(f"X_train: {X_train.shape}, y_train: {y_train_log.shape}")
print(f"X_test:  {X_test.shape},  y_test:  {y_test_real.shape}")
# Deben coincidir los primeros números (filas) de X e y

--- VERIFICACIÓN DE DIMENSIONES ---
X_train: (116, 9), y_train: (116,)
X_test:  (32, 9),  y_test:  (32,)


In [58]:
# Importación de librerías necesarias
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# import optuna
# import warnings
# warnings.filterwarnings('ignore')

# 1. CARGA LIMPIA
# try:
#     df = pd.read_parquet("data_cleaned/master_with_clusters.parquet")
# except:
#     df = pd.read_parquet("../data_cleaned/master_with_clusters.parquet")

# df['Fecha'] = pd.to_datetime(df['Fecha'])

# 2. DEFINICIÓN DE QUINCENAS (Lógica de Negocio)
# df['year'] = df['Fecha'].dt.year
# df['month'] = df['Fecha'].dt.month
# df['day'] = df['Fecha'].dt.day
# Quincena 1 (1-15), Quincena 2 (16-Fin)
# df['quincena_num'] = np.where(df['day'] <= 15, 1, 2)

# 3. EFECTO CALENDARIO (Días de Pago)
# Calculamos cuántos días de "pago fuerte" cayeron en esa quincena
# dates_ref = df[['Fecha', 'year', 'month', 'quincena_num']].drop_duplicates()
# dates_ref['es_quincena'] = dates_ref['Fecha'].dt.day.isin([15, 30, 31, 1, 25, 26, 27, 28, 29]).astype(int)
# payday_impact = dates_ref.groupby(['year', 'month', 'quincena_num'])['es_quincena'].sum().reset_index()
# payday_impact.columns = ['year', 'month', 'quincena_num', 'dias_pago_periodo']

# 4. AGRUPACIÓN BASE (Target = Ingreso)
# df_base = df.groupby(['year', 'month', 'quincena_num', 'Categoría']).agg({
#     'ingreso': 'sum',
#     'Cantidad': 'sum',
#     'Precio_Unitario': 'mean',
#     'Fecha': 'max' # Para referencia temporal
# }).reset_index()

# Unir calendario
# df_base = df_base.merge(payday_impact, on=['year', 'month', 'quincena_num'], how='left')

# print(f"Dataset Quincenal Base: {df_base.shape}")
# print(df_base.head(3))

# def create_features(df):
#     df = df.copy()
#     # Ordenar CRONOLÓGICAMENTE para que los lags tengan sentido
#     df = df.sort_values(['Categoría', 'year', 'month', 'quincena_num'])
    
#     g = df.groupby('Categoría')
    
#     # 1. TICKET PROMEDIO (Intensity)
#     df['ticket_real'] = df['ingreso'] / (df['Cantidad'] + 1)
#     df['lag_ticket_1'] = g['ticket_real'].shift(1) # ¿Cómo estuvo el ticket la quincena pasada?
    
#     # 2. PRECIO RELATIVO (Inflation/Discount)
#     # Precio actual vs promedio móvil de 2 meses (4 quincenas)
#     df['precio_movil'] = g['Precio_Unitario'].transform(lambda x: x.shift(1).rolling(4).mean())
#     df['indice_precio'] = df['Precio_Unitario'] / (df['precio_movil'] + 1e-6)
    
#     # 3. LAGS DE INGRESO (History)
#     # Lag 1 (Quincena anterior), Lag 2 (Mes anterior), Lag 4 (2 Meses)
#     for lag in [1, 2, 4]:
#         df[f'lag_ingreso_{lag}'] = g['ingreso'].shift(lag)
        
#     # 4. TENDENCIA (Momentum)
#     # Promedio móvil de ingresos (últimas 2 quincenas)
#     df['rolling_ingreso_2q'] = g['ingreso'].transform(lambda x: x.shift(1).rolling(2).mean())
    
#     # 5. CONTEXTO GLOBAL (Store Level)
#     # ¿Cómo le fue a TODA la tienda la quincena pasada?
#     # (Esto se calcula temporalmente y se une)
#     return df

# Aplicar features
# df_features = create_features(df_base)

# Contexto Global (Tienda Total)
# store_total = df_features.groupby(['year', 'month', 'quincena_num'])['ingreso'].sum().reset_index()
# store_total['store_lag_1'] = store_total['ingreso'].shift(1) # Lag global
# df_features = df_features.merge(store_total[['year', 'month', 'quincena_num', 'store_lag_1']], 
#                                 on=['year', 'month', 'quincena_num'], how='left')

# Limpieza de Nulos generados por Lags
# df_clean = df_features.dropna().reset_index(drop=True)

# One-Hot Encoding
# df_final = pd.get_dummies(df_clean, columns=['Categoría'], prefix='Cat')

# print(f"Dataset Final Listo: {df_final.shape}")

# 1. DEFINICIÓN DE COLUMNAS
# target = 'ingreso'
# cols_excluir = [
#     'ingreso', 'Cantidad', 'ticket_real', 'Precio_Unitario', # Leakage (Respuestas)
#     'Fecha', 'year', 'month', 'day', 'quincena_num', 'periodo_id' # IDs
# ]
# Seleccionar solo numéricas y quitar las prohibidas
# valid_features = df_final.select_dtypes(include=[np.number]).columns.tolist()
# features = [c for c in valid_features if c not in cols_excluir]

# 2. SPLIT TEMPORAL
# Usamos las últimas 4 quincenas (2 meses) como test
# split_date = df_final['Fecha'].max() - pd.Timedelta(weeks=8)

# train_df = df_final[df_final['Fecha'] <= split_date].reset_index(drop=True)
# test_df = df_final[df_final['Fecha'] > split_date].reset_index(drop=True)

# 3. PREPARACIÓN DE MATRICES (CRÍTICO: ALINEACIÓN)
# X_train = train_df[features]
# y_train_log = np.log1p(train_df[target]) # Log Transform

# X_test = test_df[features]
# y_test_real = test_df[target] # En pesos reales para validar

# print("--- VERIFICACIÓN DE DIMENSIONES ---")
# print(f"X_train: {X_train.shape}, y_train: {y_train_log.shape}")
# print(f"X_test:  {X_test.shape},  y_test:  {y_test_real.shape}")
# Deben coincidir los primeros números (filas) de X e y

# def objective_finance(trial):
#     # Parámetros a buscar
#     params = {
#         'objective': 'reg:squarederror', # Correcto para Log-target
#         'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
#         'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 3, 7),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         'n_jobs': -1,
#         'random_state': 42,
#         'verbosity': 0,
#         'eval_metric': "rmse",

#         'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 20, 100)
#     }
    
#     model = xgb.XGBRegressor(**params)
    
#     # Entrenar usando el Log del target
#     model.fit(
#         X_train, y_train_log,
#         eval_set=[(X_test, np.log1p(y_test_real))],
#         verbose=False,
#     )
    
#     # Predecir y devolver a escala real
#     preds_log = model.predict(X_test)
#     preds_money = np.expm1(preds_log)
#     preds_money = np.maximum(preds_money, 0)
    
#     # Calcular WMAPE
#     wmape = np.sum(np.abs(y_test_real - preds_money)) / np.sum(y_test_real)
#     return wmape

# print("Optimizando modelo financiero quincenal...")
# study = optuna.create_study(direction='minimize')
# study.optimize(objective_finance, n_trials=50)

# print("\nRESULTADOS:")
# print(f"Mejor WMAPE: {study.best_value:.1%}")
# print("Mejores parámetros:", study.best_params)

In [59]:
# ==============================================================================
# ENTRENAMIENTO FINAL Y VISUALIZACIÓN FINANCIERA
# ==============================================================================

# 1. ENTRENAR CON LOS MEJORES PARÁMETROS (LOG-SPACE)
# Parámetros encontrados por tu Optuna (WMAPE 22%)
best_params = {
    'objective': 'reg:squarederror',
    'n_estimators': 1944,
    'learning_rate': 0.096,
    'max_depth': 3, # Árboles poco profundos = Modelo robusto (menos overfitting)
    'colsample_bytree': 0.53,
    'subsample': 0.70,
    'min_child_weight': 8,
    'n_jobs': -1,
    'random_state': 42,
    'eval_metric': "rmse",

    'early_stopping_rounds': 70

}

print("Entrenando modelo financiero definitivo...")

model_fin = xgb.XGBRegressor(**best_params)

model_fin.fit(
    X_train, y_train_log,
    eval_set=[(X_train, y_train_log)],
    verbose=False
)

# 2. GENERAR PREDICCIONES
preds_log = model_fin.predict(X_test)
preds_money = np.expm1(preds_log) # Inversa Log
preds_money = np.maximum(preds_money, 0)

# Guardar en DataFrame para análisis
df_results = test.copy()
df_results['ingreso_predicho'] = preds_money
df_results['error_abs'] = np.abs(df_results['ingreso'] - df_results['ingreso_predicho'])



Entrenando modelo financiero definitivo...


In [60]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
cat_cols = [c for c in df_results.columns if c.startswith('Cat_')]
df_results['Categoría'] = df_results[cat_cols].idxmax(axis=1).str.replace('Cat_', '')
df_results = df_results.sort_values('Fecha')

# A) Datos Temporales (Quincena a Quincena)
total_forecast = df_results.groupby(['Fecha'])[['ingreso', 'ingreso_predicho']].sum().reset_index()

# B) Datos Acumulados (La "Carrera" hacia la meta)
total_forecast['ingreso_acum'] = total_forecast['ingreso'].cumsum()
total_forecast['pred_acum'] = total_forecast['ingreso_predicho'].cumsum()

# C) Datos por Categoría (Recuperando nombres si es necesario)
# (Asumimos que ya corriste el bloque anterior que recupera la columna 'Categoría')
cat_error = df_results.groupby('Categoría')[['ingreso', 'ingreso_predicho']].sum().reset_index()
cat_error['Diferencia'] = cat_error['ingreso_predicho'] - cat_error['ingreso']
cat_error['Color'] = np.where(cat_error['Diferencia'] < 0, '#e74c3c', '#27ae60') # Rojo/Verde

# 2. CREAR DASHBOARD (LAYOUT 3 PANELES)
# ------------------------------------------------------
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"colspan": 2}, None], [{}, {}]], # Fila 1 completa, Fila 2 dividida
    subplot_titles=(
        "1. Flujo de Caja Quincenal (Operativo)", 
        "2. Acumulado vs Meta (¿Llegamos al objetivo?)", 
        "3. Desvío por Categoría (Causas Raíz)"
    ),
    vertical_spacing=0.12
)

# --- GRÁFICA 1: FLUJO DE CAJA (LINEAL) ---
fig.add_trace(go.Scatter(
    x=total_forecast['Fecha'], y=total_forecast['ingreso'],
    name='Real ($)', mode='lines+markers', line=dict(color='#2c3e50', width=3)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=total_forecast['Fecha'], y=total_forecast['ingreso_predicho'],
    name='Pronóstico ($)', mode='lines+markers', line=dict(color='#3498db', width=3, dash='dash')
), row=1, col=1)

# --- GRÁFICA 2: ACUMULADO (AREA / LINEA) ---
# Esta muestra si el error se cancela o se acumula con el tiempo
fig.add_trace(go.Scatter(
    x=total_forecast['Fecha'], y=total_forecast['ingreso_acum'],
    name='Acumulado Real', mode='lines', fill='tozeroy', line=dict(color='rgba(44, 62, 80, 0.2)', width=0)
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=total_forecast['Fecha'], y=total_forecast['pred_acum'],
    name='Acumulado Pronóstico', mode='lines', line=dict(color='#e67e22', width=3)
), row=2, col=1)

# --- GRÁFICA 3: DESVÍO POR CATEGORÍA (BARRAS) ---
fig.add_trace(go.Bar(
    x=cat_error['Categoría'], y=cat_error['Diferencia'],
    marker_color=cat_error['Color'],
    text=cat_error['Diferencia'].apply(lambda x: f"${x:,.0f}"),
    textposition='auto',
    name='Diferencia ($)'
), row=2, col=2)

# 3. FORMATO FINAL EJECUTIVO
# ------------------------------------------------------
fig.update_layout(
    height=900, width=1200, 
    title_text="<b>Tablero de Control Financiero (Modelo Quincenal - WMAPE 22%)</b>",
    template="plotly_white",
    showlegend=True
)

# Formato de dinero en ejes Y
fig.update_yaxes(tickprefix="$", row=1, col=1)
fig.update_yaxes(tickprefix="$", row=2, col=1)
fig.update_yaxes(tickprefix="$", row=2, col=2)

fig.show()